# React Agent

In [1]:
from llama_index.llms.ollama import Ollama
from llama_index.core import Settings
from llama_index.embeddings.ollama import OllamaEmbedding

In [2]:
Settings.llm = Ollama(model = "qwen2.5:3b")
Settings.embed_model = OllamaEmbedding(model_name="nomic-embed-text")

In [1]:
import pandas as pd
from PyPDF2 import PdfReader, PdfWriter
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
import os

def fill_certificate(excel_file, pdf_template, output_folder):
    df = pd.read_excel(excel_file)
    
    os.makedirs(output_folder, exist_ok=True)
    
    if 'Name' not in df.columns or 'Department' not in df.columns:
        raise ValueError("Excel file must contain 'Name' and 'Department' columns")
    
    reader = PdfReader(pdf_template)
    
    for index, row in df.iterrows():
        name = row['Name']
        department = row['Department']
        
        writer = PdfWriter()
        
        overlay_path = "overlay.pdf"
        page_width = float(reader.pages[0].mediabox[2])
        page_height = float(reader.pages[0].mediabox[3])

        c = canvas.Canvas(overlay_path, pagesize=(page_width, page_height))
        
        name_x, name_y = 360, 280  
        dept_x, dept_y = 680, 280  
        
        c.setFont("Times-BoldItalic", 18)  
        c.setFillColorRGB(0.55, 0.0, 0.0)  
        c.drawString(dept_x, dept_y, department)
        c.drawString(name_x, name_y, name)

        c.save()
        
        overlay_reader = PdfReader(overlay_path)
        for page in reader.pages:
            page.merge_page(overlay_reader.pages[0])
            writer.add_page(page)
        
        output_path = os.path.join(output_folder, f"{name}.pdf")
        with open(output_path, "wb") as output_pdf:
            writer.write(output_pdf)
    
    print("Certificates generated successfully!")

fill_certificate('data.xlsx', 'template.pdf', 'results')


Certificates generated successfully!


In [1]:
import pandas as pd
from PyPDF2 import PdfReader, PdfWriter
from reportlab.pdfgen import canvas
import os

def fill_certificate(excel_file, pdf_template, output_folder):
    df = pd.read_excel(excel_file)
    
    os.makedirs(output_folder, exist_ok=True)
    
    if 'Name' not in df.columns or 'Department' not in df.columns:
        raise ValueError("Excel file must contain 'Name' and 'Department' columns")
    
    for index, row in df.iterrows():
        name = str(row['Name']).strip()
        department = str(row['Department']).strip()
        
        template_reader = PdfReader(pdf_template)
        writer = PdfWriter()

        # Generate a unique overlay file for each entry
        overlay_path = f"overlay_{index}.pdf"
        page_width = float(template_reader.pages[0].mediabox[2])
        page_height = float(template_reader.pages[0].mediabox[3])

        # Create the overlay PDF with the participant's name and department
        c = canvas.Canvas(overlay_path, pagesize=(page_width, page_height))
        
        name_x, name_y = 360, 280  
        dept_x, dept_y = 680, 280  

        c.setFont("Times-BoldItalic", 18)  
        c.setFillColorRGB(0.55, 0.0, 0.0)  
        c.drawString(dept_x, dept_y, department)
        c.drawString(name_x, name_y, name)

        c.save()

        overlay_reader = PdfReader(overlay_path)
        overlay_page = overlay_reader.pages[0]

        # Get only the first page of the template and merge the overlay
        template_page = template_reader.pages[0]
        template_page.merge_page(overlay_page)
        writer.add_page(template_page)

        # Save the output certificate
        output_path = os.path.join(output_folder, f"{name}.pdf")
        with open(output_path, "wb") as output_pdf:
            writer.write(output_pdf)
        
        # Remove overlay file after processing
        os.remove(overlay_path)

    print("Certificates generated successfully!")

fill_certificate('data.xlsx', 'template.pdf', 'results')


Certificates generated successfully!
